In [ ]:
import pandas as pd
import json
import requests
from pymongo import MongoClient
from bson.objectid import ObjectId
import os
from pprint import pprint
import datetime as dt

In [ ]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\mongodb_api.txt','r') as f:
    cluster = f.readlines()[0]
    
client = MongoClient(cluster)

# print(client.list_database_names())

db = client.FinanceApp
balance_sheet_collection = db.balance_sheet
income_collection = db.income_statement
cash_collection = db.cash_flow_statement
company_profile = db.company_profile
historical = db.historical
stock_split = db.stock_split

In [ ]:
arr = os.listdir(r"D:\\lianz\Desktop\\Python\\personal_projects\\personal_finance\\balance-sheet-statement")
len(arr)

In [ ]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\fmp_api.txt','r') as f:
    fmp_api = f.readlines()[0]

In [ ]:
def select_profile(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?apikey={fmp_api}")
    r = r.json()
    return r

In [ ]:
a_profile = select_profile('A','profile')

In [ ]:
a_profile[0]['symbol']

In [ ]:
company_profile.insert_one(a_profile[0])

In [ ]:
from pymongo import ASCENDING, DESCENDING

In [ ]:
a_data = [i for i in balance_sheet_collection.find({'symbol':'AAL'}).sort('date', DESCENDING)]

In [ ]:
a_data

In [ ]:
company_profile.delete_one({})

##### Define function to access elements of an entry in database
Use function to access individual elements of a document in a collection

In [ ]:
def access_entry(collection_name, entry_name, entry_value, return_value):
    data = collection_name.find({entry_name:entry_value})

    data = [i[return_value] for i in data]

    return data

##### Get list of tickers in database

In [ ]:
balance_list = list(set([i['symbol'] for i in balance_sheet_collection.find()]))
income_list = list(set([i['symbol'] for i in income_collection.find()]))
cash_list = list(set([i['symbol'] for i in cash_collection.find()]))
company_list = list(set([i['symbol'] for i in company_profile.find()]))
list_tickers = [balance_list, income_list, cash_list, company_list]
lengths = [len(i) for i in list_tickers]
missing_tickers = [i for i in list_tickers if len(i) == max(lengths)]

In [ ]:
profiles = [select_profile(i,'profile') for i in balance_list]

In [ ]:
downloaded = [i[0]['symbol'] for i in profiles if len(i)>0]

In [ ]:
def select_quote(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?limit=120&apikey=eb29218df82acef0486b5c014ccec868")
    r = r.json()
    return r

##### Insert to database from API

In [ ]:
def insert_to_mongoDB(ticker):

    balance = select_quote(ticker, 'balance-sheet-statement')
    income = select_quote(ticker, 'income-statement')
    cash_flow = select_quote(ticker, 'cash-flow-statement')

    if len(balance) <= 1 or len(income) <= 1 or len(cash_flow) <= 1:
        pass
    else:
        for i in balance:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        for i in income:
            i['index_id'] = f"{i['symbol']}_{i['date']}"    
        for i in cash_flow:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        
        bal_ids = [i['index_id'] for i in balance if i['index_id'] not in access_entry(balance_sheet_collection,'symbol',i['symbol'],'index_id')]
        income_ids = [i['index_id'] for i in income if i['index_id'] not in access_entry(income_collection,'symbol',i['symbol'],'index_id')]
        cash_ids = [i['index_id'] for i in cash_flow if i['index_id'] not in access_entry(cash_collection,'symbol',i['symbol'],'index_id')]

        try:
            balance_sheet_collection.insert_many([i for i in balance if i['index_id'] in bal_ids])
            income_collection.insert_many([i for i in income if i['index_id'] in income_ids])
            cash_collection.insert_many([i for i in cash_flow if i['index_id'] in cash_ids])
        except:
            pass

##### Insert to database from API per collection

In [ ]:
def insert_to_mongoDB(collection, ticker, statement):

    statement = select_quote(ticker, statement)

    if len(statement) <= 1:
        pass
    else:
        for i in statement:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        
        ids = [i['index_id'] for i in statement if i['index_id'] not in access_entry(collection,'symbol',i['symbol'],'index_id')]

        try:
            collection.insert_many([i for i in statement if i['index_id'] in ids])
        except:
            print('statement already exists.')

In [ ]:
insert_to_mongoDB(balance_sheet_collection, 'AAL', 'balance-sheet-statement')
insert_to_mongoDB(cash_collection, 'AAL', 'cash-flow-statement')
insert_to_mongoDB(income_collection, 'AAL', 'income-statement')

In [ ]:
# balance_sheet_collection.create_index('index_id', unique=True)
# cash_collection.create_index('index_id', unique=True)
# income_collection.create_index('index_id', unique=True)
# company_profile.create_index('index_id', unique=True)
# historical.create_index('index_id', unique=True)
stock_split.create_index('index_id', unique=True)

##### Insert to database from local computer

In [ ]:
def insert_to_mongoDB(ticker):

    balance = f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\balance-sheet-statement\\{ticker}'
    income = f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\income-statement\\{ticker}'
    cash_flow = f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\cash-flow-statement\\{ticker}'

    with open(balance, 'r') as f:
        data_bal = json.load(f)
    with open(income, 'r') as f:
        data_income = json.load(f)
    with open(cash_flow, 'r') as f:
        data_cash = json.load(f)


    if len(data_bal) <= 1 or len(data_income) <= 1 or len(data_cash) <= 1:
        pass
    else:
        for i in data_bal:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        for i in data_income:
            i['index_id'] = f"{i['symbol']}_{i['date']}"    
        for i in data_cash:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        
        bal_ids = [i['index_id'] for i in data_bal if i['index_id'] not in access_entry(balance_sheet_collection,'symbol',i['symbol'],'index_id')]
        income_ids = [i['index_id'] for i in data_bal if i['index_id'] not in access_entry(income_collection,'symbol',i['symbol'],'index_id')]
        cash_ids = [i['index_id'] for i in data_bal if i['index_id'] not in access_entry(cash_collection,'symbol',i['symbol'],'index_id')]
        
        try:
            balance_sheet_collection.insert_many([i for i in data_bal if i['index_id'] in bal_ids])
            income_collection.insert_many([i for i in data_income if i['index_id'] in income_ids])
            cash_collection.insert_many([i for i in data_cash if i['index_id'] in cash_ids])
        except:
            pass

##### Upload all currently stored statements to cloud database

In [ ]:
for i in arr:
    insert_to_mongoDB(i)

In [ ]:
# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

##### Delete database entries (only run when needed)

In [ ]:
balance_sheet_collection.delete_many({'symbol':'AF'})
cash_collection.delete_many({'symbol':'AF'})
income_collection.delete_many({'symbol':'AF'})

In [ ]:
balance_sheet_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})
cash_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})
income_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})

In [ ]:
A_2022 = balance_sheet_collection.find({'symbol':'A'})

In [ ]:
[i for i in A_2022]

In [ ]:
balance_sheet_collection.create_index("index_id", unique=True)

In [ ]:
with open('D:\lianz\Desktop\Python\personal_projects\\finance_dashboard\\fmp_api.txt','r') as f:
    fmp_api = f.readlines()[0]

In [ ]:
r = requests.get(
    f"https://financialmodelingprep.com/api/v3/historical-price-full/CVX?apikey={fmp_api}")

r = r.json()

In [ ]:
def access_entry(_collection_name, entry_name, entry_value, return_value):
    data = _collection_name.find({entry_name: entry_value})

    data = [i[return_value] for i in data]

    return data

In [ ]:
for i in r['historical']:    
    i['index_id'] = f"{r['symbol']}_{i['date']}"
    i['symbol'] = f"{r['symbol']}"

ids = [i['index_id'] for i in r['historical'] if i['index_id']
            not in access_entry(historical, 'symbol', 'AAPL', 'index_id')]
ids

In [ ]:
try:
    historical.insert_many([i for i in r['historical'] if i['index_id'] in ids])
    print(f"{r['symbol']} historical updated!")

except:
    print(f"{r['symbol']} historical already exists")

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = pd.DataFrame.from_records([x for i,x in enumerate(r['historical'])], index='date')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=df.index, y=df['adjClose'], mode="lines+markers", name=f"adjClose"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(
        x=df.index, y=df['volume'], marker=dict({'color': 'darkorange'}), texttemplate="%{value:,}", textposition="inside", name="Volume"),
    secondary_y=True,
)


In [ ]:
r['historical'][0].keys()

In [212]:
aapl_hist = pd.DataFrame.from_records([x for i,x in enumerate(historical.find({'symbol':'AAPL'}))], index='date').sort_index()

In [222]:
aapl_hist.loc[aapl_hist.index.date >= dt.date(2023,1,1)].index

DatetimeIndex(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-17', '2023-01-18', '2023-01-19',
               '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25',
               '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31',
               '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06',
               '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10',
               '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16',
               '2023-02-17', '2023-02-21', '2023-02-22', '2023-02-23',
               '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01',
               '2023-03-02', '2023-03-03', '2023-03-07'],
              dtype='datetime64[ns]', name='date', freq=None)

In [ ]:
def download_stocksplit(ticker):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_split/{ticker}?apikey={fmp_api}"
    )
    r = r.json()
    return r

In [ ]:
aapl_stocksplit = download_stocksplit('AAPL')

In [ ]:
for i in aapl_stocksplit['historical']:
    i['index_id'] = f"{aapl_stocksplit['symbol']}_{i['date']}"
    i['date'] = dt.datetime.strptime(i['date'], '%Y-%m-%d')

In [ ]:
pd.DataFrame.from_records(aapl_stocksplit['historical'], index='date')

In [ ]:
def stock_price_api(ticker):
    url = "https://alpha-vantage.p.rapidapi.com/query"
    headers = {"X-RapidAPI-Key": "4cf7dba8camshc88301e2a919dd8p18fae9jsn2c1f30ad5ee4",
               "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com"}
    querystring = {"function":"TIME_SERIES_DAILY","symbol":f"{ticker}","outputsize":"full","datatype":"json"}
    response = requests.request("GET", url=url, headers=headers, params=querystring)
    return response.json()

In [ ]:
historical.delete_many({'symbol':'A'})

In [ ]:
file = stock_price_api('A')


In [ ]:
for i,x in file['Time Series (Daily)'].items():    
    x['index_id'] = f"{'A'}_{i}"
    x['symbol'] = f"{'A'}"
    x['date'] = i
historical.insert_many([x for i,x in file['Time Series (Daily)'].items()])

In [ ]:
historical.update_many({'date':{'$type':"string"}},[
    {
        '$set': {
            'date': {
                '$dateFromString': {
                    'dateString': '$date', 
                    'format': '%Y-%m-%d'
                }
            }
        }
    }
])

In [223]:
list(set([i['index_id'] for i in historical.find()]))

KeyError: 'index_id'